In [98]:
import sys
import pandas as pd
import warnings
import sqlite3
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
warnings.filterwarnings('ignore')


In [99]:
def load_data(file_path):
    """
    :param file_path: This variable takes in the filepath of the database to read.
    :return: The function returns the table as a dataframe

    """

    conn = sqlite3.connect(file_path)

    df = pd.read_sql_query("SELECT * from Disaster_data", con=conn)

    X = df['message'].values
    Y = df.iloc[:, 4:]
    category_names = Y.columns

    conn.close()

    return X, Y, category_names


In [114]:
def tokenize(text):
    """
    :param text: takes in a string value and using nltk methods normalizes,tokenizes and lemmatizes it.
    :return:
    
    """

    # Find all urls if any exists in the text and replace it with the word 'url_placeholder'
    url_format = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    all_urls = re.findall(url_format, text)

    for u in all_urls:
        text = text.replace(u, 'url_placeholder')

    # Tokenize the text
    tokenized_text = word_tokenize(text.lower())

    # Lemmatize the text
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []

    for token in tokenized_text:
        lemmatized_text = lemmatizer.lemmatize(token).strip()
        clean_tokens.append(lemmatized_text)

    return clean_tokens


In [ ]:
def build_model():
    pass
